In [ ]:
import zipfile
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from collections import Counter
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
filename = "given_dataset.csv.zip"
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
file = "given_dataset.csv"
df = pd.read_csv(file)

In [ ]:
# from sklearn.preprocessing import KBinsDiscretizer
data = df.to_numpy()
print(data.shape)
Y = data[:, -1]
X = data[:, 1:8]
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

(2041687, 9)


In [ ]:
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)
cnt = Counter(Ytrain)

(1367930, 7) (673757, 7) (1367930,) (673757,)


Counter({0.0: 19483, 1.0: 1348447})

In [ ]:
decision_tree = DecisionTreeClassifier( max_leaf_nodes = 25, max_depth = 15, class_weight='balanced',
                                       max_features=7, random_state=20, criterion='gini', min_samples_leaf=20)

decision_tree.fit(X, Y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=15, max_features=7, max_leaf_nodes=25,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=20, splitter='best')

In [ ]:
prediction = decision_tree.predict(Xtest)
roc_auc_score(Ytest, prediction)

0.6945315493669406

In [ ]:
confusion_matrix(Ytest, prediction)

array([[  6732,   2795],
       [210933, 453297]])

In [ ]:
from sklearn.ensemble import BaggingClassifier
bag_tree = BaggingClassifier(
    DecisionTreeClassifier( max_leaf_nodes = 25, max_depth = 15, class_weight='balanced',
                                       max_features=7, random_state=20, criterion='gini', min_samples_leaf=20),
                             oob_score= True, verbose = 1, bootstrap_features = True, n_jobs = 4, n_estimators = 10
)  
bag_tree.fit(X, Y)

In [ ]:
prediction = bag_tree.predict(Xtest)
roc_auc_score(Ytest, prediction)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.8s finished


0.7307596362426932

In [ ]:
confusion_matrix(Ytest, prediction)

array([[  6687,   2840],
       [159668, 504562]])

In [ ]:
BaggingClassifier()

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier

sm = SMOTE(random_state=27, ratio=1)
Xtraine, Ytraine = sm.fit_sample(Xtrain, Ytrain)

# ada_boost = AdaBoostClassifier(n_estimators=50, random_state=0)
# ada_boost.fit(Xtraine, Ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# prediction = ada_boost.predict(Xtest)
# roc_auc_score(Ytest, prediction)
Counter(Ytrain)

Counter({0.0: 337111, 1.0: 1348447})

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier

bal_tree = BalancedBaggingClassifier( 
        DecisionTreeClassifier(max_depth=25, random_state=20, min_samples_leaf=20),
        oob_score= False, verbose = 1, bootstrap_features = True, n_jobs = 8, n_estimators = 51, warm_start = False) 

        # DecisionTreeClassifier(max_depth= 26, random_state=20, min_samples_leaf=20),
        # oob_score= True, verbose = 1, bootstrap_features = True, n_jobs = 6, n_estimators = 61)

bal_tree.fit(Xtraine, Ytrain)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:  5.2min remaining: 10.3min


In [ ]:
prediction = bal_tree.predict(Xtest)
roc_auc_score(Ytest, prediction)

# DecisionTreeClassifier(max_depth=24, random_state=20, min_samples_leaf=20),
        # oob_score= True, verbose = 1, bootstrap_features = True, n_jobs = 6, n_estimators = 35)
        # 0.7856892081999828

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    5.9s remaining:   11.8s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    7.6s finished


0.7827742255032519

In [ ]:
confusion_matrix(Ytest, prediction)

array([[  7034,   2493],
       [109827, 554403]])

In [ ]:
RandomForestClassifier()

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
balencedRF = BalancedRandomForestClassifier(max_depth=50,random_state=0)
balencedRF.fit(Xtrain, Ytrain)

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier
balencedrus = EasyEnsembleClassifier(  )
balencedrus.fit(Xtrain, Ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

EasyEnsembleClassifier(base_estimator=None, n_estimators=10, n_jobs=1,
                       random_state=None, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [ ]:
prediction = balencedrus.predict(Xtest)
roc_auc_score(Ytest, prediction)

0.6960817508050705

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarining)

In [ ]:
pred_filename = "to_predict.csv.zip"
with zipfile.ZipFile(pred_filename, 'r') as zip_ref:
    zip_ref.extractall()
file_name = "to_predict.csv"
df_pred = pd.read_csv(file_name)


In [ ]:
prediction = bal_tree.predict(df_pred.to_numpy()[:, 1:])
df_pred = pd.DataFrame(data = prediction,  
                  index = df_pred.iloc[ : ,0]) 
df_pred.rename(columns={0: "T"}, inplace = True)
df_pred.to_csv('output.csv')

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    3.5s remaining:    7.1s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    4.5s finished


In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_classes=3,
                           n_informative=4, weights=[0.2, 0.3, 0.5],
                           random_state=0)

In [ ]:
X.shape

(1000, 20)

In [ ]:
# from imblearn.ensemble import BalancedRandomForestClassifier
# balencedRF = BalancedRandomForestClassifier(max_depth=50,random_state=0)
# balencedRF.fit(X, y)

In [ ]:
# rfclass = RandomForestClassifier(random_state=0, n_jobs=4, n_estimators=43, 
#                                  class_weight="balanced_subsample", max_depth=14, verbose = 1)

rfclass = RandomForestClassifier(random_state=0, max_depth = 20, class_weight={0: 100, 1: 1}, 
                                 n_jobs=4, n_estimators= 101,verbose = 1)
rfclass.fit(X_res, y_res)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.8s
[Parallel(n_jobs=4)]: Done 101 out of 101 | elapsed:    4.2s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 100, 1: 1}, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=101, n_jobs=4, oob_score=False,
                       random_state=0, verbose=1, warm_start=False)

In [ ]:
prediction = rfclass.predict(Xtest)
roc_auc_score(Ytest, prediction)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 101 out of 101 | elapsed:    7.8s finished


0.6441801887294092

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# max_depth = 17
rfreg = RandomForestRegressor(n_estimators = 500, max_depth = 17, oob_score= True,
                              n_jobs = 4, verbose = 1)
rfreg.fit(X_res, y_res)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   17.8s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   40.7s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   45.9s finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=17, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=4, oob_score=True,
                      random_state=None, verbose=1, warm_start=False)

In [ ]:
prediction = rfreg.predict(Xtest)
roc_auc_score(Ytest, prediction)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   12.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   28.2s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   31.8s finished


0.8609428979925933

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Counter(y_res)

Counter({0.0: 19483, 1.0: 19483})

In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='0.5')
X_res1, y_res1 = oversample.fit_resample(Xtrain, Ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Counter(y_res1)

Counter({0.0: 1348447, 1.0: 1348447})

In [ ]:
from xgboost import XGBClassifier
# 20 350
model = XGBClassifier(n_jobs = 4, max_depth = 38, 
                      n_estimators=450, verbosity=0)
model.fit(X_res, y_res)

In [ ]:
prediction = model.predict_proba(Xtest)
pred = prediction[:, 1]
roc_auc_score(Ytest, pred)

0.8695870332695581

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# 7, 300
# 0.8800296303204439 8 - 300
# gradboost = GradientBoostingClassifier(max_depth = 8, n_estimators = 250, 
#                                        validation_fraction= 0.2, subsample= 0.8, 
#                                        max_features = 'auto', 
#                                        warm_start = True, min_samples_split = 3)
# gradboost.fit(X_res, y_res)

from sklearn.ensemble import GradientBoostingRegressor
gradboostreg = GradientBoostingRegressor(max_depth = 8, n_estimators = 350, 
                                        validation_fraction= 0.25, 
                                        subsample = 0.9
                                       )
gradboostreg.fit(X_res, y_res)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=8,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=350,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=0.9, tol=0.0001,
                          validation_fraction=0.25, verbose=0,
                          warm_start=False)

In [ ]:
# prediction = gradboost.predict_proba(Xtest)
# pred = prediction[:, 1] 

prediction = gradboostreg.predict(Xtest)
roc_auc_score(Ytest, prediction)
# 0.8811133892182161

0.8771125266933775

In [ ]:
from xgboost import XGBRegressor

# 9 350
# 0.882
# 88.257
model = XGBRegressor(n_jobs = 4,max_depth =9,objective='reg:logistic',learning_rate=0.09, subsample=0.9,n_estimators=5000,
                     reg_alpha=0.005, min_child_weight=6,colsample_bytree=0.9,seed=27)
model.fit(X_res, y_res)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0,
             importance_type='gain', learning_rate=0.09, max_delta_step=0,
             max_depth=9, min_child_weight=6, missing=None, n_estimators=5000,
             n_jobs=4, nthread=None, objective='reg:logistic', random_state=0,
             reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, seed=27,
             silent=None, subsample=0.9, verbosity=1)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.2)
X_res, y_res = rus.fit_resample(Xtrain, Ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from xgboost import XGBRegressor
# 9 350
# 0.882
# 88.257
model = XGBRegressor(n_jobs = 4,max_depth =9,objective='reg:logistic',learning_rate=0.09, subsample=0.9,n_estimators=300)
model.fit(X_res, y_res)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.09, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=300,
             n_jobs=4, nthread=None, objective='reg:logistic', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=1)

In [ ]:
prediction = model.predict(Xtest)
roc_auc_score(Ytest, prediction)

0.8881283625247471

In [ ]:
pred_filename = "to_predict.csv.zip"

with zipfile.ZipFile(pred_filename, 'r') as zip_ref:
    zip_ref.extractall()

file_name = "to_predict.csv"
df_pred = pd.read_csv(file_name)


In [ ]:
prediction = model.predict(df_pred.to_numpy()[:, 1:])

# pred = prediction[:, 1]

In [ ]:
# prediction.shape
df_pred = pd.DataFrame(data = prediction,  
                  index = df_pred.iloc[ : ,0]) 

In [ ]:
df_pred.rename(columns={0: "T"}, inplace = True)

In [ ]:
df_pred.to_csv('output.csv')

In [ ]:
import pickle
pickle.dump(model,open('model_xgbregressor_best','wb'))

NameError: ignored

In [ ]:
import pickle
pickle.dump(model,open('model_xgbregressor2','wb'))